# [Data Cleaning and Preperation](https://wesmckinney.com/book/data-cleaning.html)

Listen, it's natural that you will run into missing data. During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up 80% or more of an analyst's time.

And sometimes that Data is missing! How _fun_, but that does mean we need a way to handle it.

thankfully, Panda's has you covered.

# 7.1 Handling Missing Data

all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect, but it is sufficient for most real-world use. For data with `float64` dtype, pandas uses the floating-point value `NaN` (Not a Number) to represent missing data.

We call this a ___sentinel value___: when present, it indicats a missing or _null_ value

In [2]:
import pandas as pd
import numpy as np
float_data = pd.Series([1.2, -3.5, np.nan, 0])

float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

The `isna` method gives us a Boolean Series with `True` where values are null:

In [3]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we've adopted a convention used in the R programming language by referring to missing data as NA, which stands for not available. In statistics applications, NA data may either be data that __does not exist__ or __that exists but was not observed (through problems with data collection, for example)__. When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

Python as `None`, and that is treated as NA:

In [4]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

string_data


0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [5]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [6]:
float_data = pd.Series([1, 2, None], dtype='float64')

float_data

0    1.0
1    2.0
2    NaN
dtype: float64

In [7]:
float_data.isna()

0    False
1    False
2     True
dtype: bool

pandas has attempted to make working with missing data consistent across data types. Functions like `pandas.isna` abstract away many of the annoying details. Here are some more [Na Handling object methods](https://wesmckinney.com/book/data-cleaning.html#tbl-table_na_method)

## Filtering Out Missing Data

There are a few ways to filter out missing data. While you can use `pandas.isna` and Boolean indexing `dropna` can be helpful. On a Series, it returns the Series with only the nonnull data and index values:

In [8]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is the same as the following:

In [9]:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, there are different ways to remove missing data. You may want to drop rows or columns that are all NA, or only those rows or columns containing any NAs at all. `dropna` by default drops __any row containing a missing value__:



In [10]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data


,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Passing `how="all"` will drop only rows that are all NA:


In [12]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Keep in mind that these functions return new objects by default and do not modify the contents of the original object.

To drop columns in the same way, pass `axis="columns"`:



In [13]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [14]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Suppose you want to keep __only rows containing at most a certain number of missing observations__. You can indicate this with the `thresh` argument:



In [15]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df


,0,1,2
0,0.849322,NaN,NaN
1,-0.034933,NaN,NaN
2,1.569254,NaN,-0.005351
3,-1.385123,NaN,-1.199080
4,0.651837,0.708555,0.617525
5,1.342012,0.150200,1.066098
6,0.518509,-0.996391,0.622795


In [16]:
df.dropna()


,0,1,2
4,0.651837,0.708555,0.617525
5,1.342012,0.150200,1.066098
6,0.518509,-0.996391,0.622795


In [17]:
df.dropna(thresh=2)

,0,1,2
2,1.569254,NaN,-0.005351
3,-1.385123,NaN,-1.199080
4,0.651837,0.708555,0.617525
5,1.342012,0.150200,1.066098
6,0.518509,-0.996391,0.622795


## Filter in Missing Data
Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways. For most purposes, the `fillna` method is the workhorse function to use. Calling `fillna` with a constant replaces missing values with that value:|



In [18]:
df.fillna(0)

,0,1,2
0,0.849322,0.000000,0.000000
1,-0.034933,0.000000,0.000000
2,1.569254,0.000000,-0.005351
3,-1.385123,0.000000,-1.199080
4,0.651837,0.708555,0.617525
5,1.342012,0.150200,1.066098
6,0.518509,-0.996391,0.622795


Calling `fillna` with a dictionary, you can use a different fill value for each column:



In [19]:
df.fillna({1: 0.5, 
           2: 0})

,0,1,2
0,0.849322,0.500000,0.000000
1,-0.034933,0.500000,0.000000
2,1.569254,0.500000,-0.005351
3,-1.385123,0.500000,-1.199080
4,0.651837,0.708555,0.617525
5,1.342012,0.150200,1.066098
6,0.518509,-0.996391,0.622795


The same intrerpolation methods available for reindexing can be used with `fillna`:

In [20]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,0,1,2
0,1.022788,1.179079,1.259649
1,-2.140086,2.438710,1.679905
2,0.433713,NaN,1.280977
3,1.400983,NaN,-0.716727
4,0.614216,NaN,NaN
5,0.189180,NaN,NaN


In [21]:
df.fillna(method="ffill")

,0,1,2
0,1.022788,1.179079,1.259649
1,-2.140086,2.438710,1.679905
2,0.433713,2.438710,1.280977
3,1.400983,2.438710,-0.716727
4,0.614216,2.438710,-0.716727
5,0.189180,2.438710,-0.716727


In [22]:
df.fillna(method="ffill", limit=2)



,0,1,2
0,1.022788,1.179079,1.259649
1,-2.140086,2.438710,1.679905
2,0.433713,2.438710,1.280977
3,1.400983,2.438710,-0.716727
4,0.614216,NaN,-0.716727
5,0.189180,NaN,-0.716727


With `fillna` you can do lots of other things such as simple data imputation using the median or mean statistics:



In [23]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

[`fillna`](https://wesmckinney.com/book/data-cleaning.html#tbl-table_fillna_function) function arguments

# Data Transformation

So far we have been concerned about handling missing data. Filtering, cleaning, and other transformations are another class of important operations.

## Removing Duplications
Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [24]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method `duplicated` returns a Boolean Series indicating whether each row is a duplicate (its column values are exactly equal to those in an earlier row) or not:



In [25]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, `drop_duplicates` returns a DataFrame with rows where the `duplicated` array is `False` filtered out:



In [26]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates based only on the `"k1"` column:



In [27]:
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [28]:
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by default keep the first observed value combination. Passing `keep="last"` will return the last one:



In [29]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping
For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:



In [30]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food came from. Let’s write down a mapping of each distinct meat type to the kind of animal:



In [31]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

The `map` method on a Series (also discussed in [Ch 5.2.5: Function Application and Mapping](https://wesmckinney.com/book/pandas-basics.html#pandas_apply)) accepts a function or dictionary-like object containing a mapping to do the transformation of values:


In [32]:
data["animal"] = data["food"].map(meat_to_animal)
data


,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work:



In [33]:
def get_animal(x):
    return meat_to_animal[x]

data["food"].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using `map` is a convenient way to perform element-wise transformations and other data cleaning-related operations.



## Replacing Values
Filling in missing data with the `fillna` method is a special case of more general value replacement. As you've already seen, `map` can be used to modify a subset of values in an object, but `replace` provides a simpler and more flexible way to do so. Let’s consider this Series:



In [34]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The `-999` values might be sentinel values for missing data. To `replace` these with NA values that pandas understands, we can use replace, producing a new Series:



In [35]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

You can replace multiple values at once by passing a list and then the substitue value:

In [36]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:

In [37]:
data # A reminder of what we are messing with

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [38]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dictionary:

In [39]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

 - The `data.replace` method is distinct from `data.str.replace`, which performs element-wise string substitution. We look at these string methods on Series later in the chapter.

## Renaming Axis Indexes
Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in place without creating a new data structure. Here’s a simple example:



In [40]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])

Like a Series, the axis indexes have a `map` method:

In [41]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

You can assign to the `index` attribute, modifying the DataFrame in place:



In [42]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


If you want to create a transformed version of a dataset without modifying the original, a useful method is `rename`:



In [43]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Notably, `rename` can be used in conjunction with a dictionary-like object, providing new values for a subset of the axis labels:



In [44]:
data.rename(index={"OHIO": "INDIANA"},
            columns={"three": "peekaboo"})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


`rename` saves you from the chore of copying the DataFrame manually and assigning new values to its `index` and `columns` attributes.

# Discretization and Binning

Continuous data is often discretized or otherwise seperated into "bins" for analysis, say you have a group of people in a study, and you want to group them into discrete age buckets.



In [45]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you have to use `pandas.cut`:



In [46]:
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special Categorical object. The output you see describes the bins computed by `pandas.cut`. Each bin is identified by a special (unique to pandas) interval value type containing the lower and upper limit of each bin:



In [47]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [48]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [49]:
age_categories.categories[0]

Interval(18, 25, closed='right')

In [50]:
pd.value_counts(age_categories)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

Note that `pd.value_counts(categories)` are the bin counts for the result of `pandas.cut`.

In the string representation of an interval, a parenthesis means that the side is open (exclusive), while the square bracket means it is closed (inclusive). You can change which side is closed by passing `right=False`:



In [51]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

You can override the default interval-based bin labeling by passing a list or array to the `labels` option:



In [52]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to `pandas.cut` instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data. Consider the case of some uniformly distributed data chopped into fourths:


In [53]:
data = np.random.uniform(size=20)
pd.cut(data, 4, precision=2)

[(0.74, 0.98], (0.27, 0.5], (0.27, 0.5], (0.028, 0.27], (0.74, 0.98], ..., (0.028, 0.27], (0.028, 0.27], (0.74, 0.98], (0.028, 0.27], (0.27, 0.5]]
Length: 20
Categories (4, interval[float64, right]): [(0.028, 0.27] < (0.27, 0.5] < (0.5, 0.74] < (0.74, 0.98]]

The `precision=2` option limits the decimal precision to two digits.

A closely related function, `pandas.qcut`, bins the data based on sample quantiles. Depending on the distribution of the data, using `pandas.cut` will not usually result in each bin having the same number of data points. Since `pandas.qcut` uses sample quantiles instead, you will obtain roughly equally sized bins:

In [54]:
data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(0.7, 3.15], (-0.65, 0.016], (-0.65, 0.016], (0.016, 0.7], (-0.65, 0.016], ..., (0.016, 0.7], (-0.65, 0.016], (-0.65, 0.016], (0.7, 3.15], (-3.5, -0.65]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.5, -0.65] < (-0.65, 0.016] < (0.016, 0.7] < (0.7, 3.15]]

In [55]:
pd.value_counts(quartiles)

(-3.5, -0.65]     250
(-0.65, 0.016]    250
(0.016, 0.7]      250
(0.7, 3.15]       250
dtype: int64

Similar to `pandas.cut`, you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [56]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-3.496, -1.224]    100
(-1.224, 0.016]     400
(0.016, 1.303]      400
(1.303, 3.151]      100
dtype: int64

We’ll return to `pandas.cut` and `pandas.qcut` later in the chapter during our discussion of aggregation and group operations, as these discretization functions are especially useful for quantile and group analysis.

## Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [57]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.000365,0.080913,-0.020961,0.051949
std,1.014592,0.961590,0.964079,0.973771
min,-2.808034,-3.212967,-3.539944,-2.941416
25%,-0.729234,-0.557616,-0.676983,-0.606048
50%,0.030599,0.082004,-0.010033,0.044953
75%,0.683587,0.708122,0.645752,0.705190
max,3.462372,2.869535,2.875731,2.969455


Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:



In [58]:
col = data[2]
col[col.abs() > 3]

193   -3.049075
869   -3.539944
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the `any` method on a Boolean DataFrame:



In [59]:
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
193,-0.634253,-0.037746,-3.049075,0.607270
585,0.093468,-3.212967,1.147746,-0.015665
629,3.462372,0.659477,-1.053553,-0.193315
869,-0.446970,-0.195543,-3.539944,-1.592781
961,3.004762,-0.559462,0.131920,0.910826


The parentheses around `data.abs() > 3` are necessary in order to call the `any` method on the result of the comparison operation.

Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3:



In [60]:
data[data.abs() > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.000102,0.081126,-0.020372,0.051949
std,1.013103,0.960884,0.962102,0.973771
min,-2.808034,-3.000000,-3.000000,-2.941416
25%,-0.729234,-0.557616,-0.676983,-0.606048
50%,0.030599,0.082004,-0.010033,0.044953
75%,0.683587,0.708122,0.645752,0.705190
max,3.000000,2.869535,2.875731,2.969455


The statement `np.sign(data)` produces 1 and –1 values based on whether the values in `data` are positive or negative:



In [61]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,-1.0
1,1.0,-1.0,-1.0,-1.0
2,1.0,1.0,1.0,-1.0
3,1.0,1.0,-1.0,1.0
4,1.0,-1.0,1.0,-1.0


## Permutation and Random Sampling
Permuting (randomly reordering) a Series or the rows in a DataFrame is possible using the `numpy.random.permutation` function. Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering:



In [62]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [63]:
sampler = np.random.permutation(5)
sampler


array([0, 1, 4, 2, 3])

That array can then be used in `iloc`-based indexing or the equivalent `take` function:

In [64]:
df.take(sampler)

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27


In [65]:
df.iloc[sampler]

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27


By invoking `take` with `axis="columns"`, we could also select a permutation of the columns:

In [66]:
column_sampler = np.random.permutation(7)
column_sampler

array([0, 1, 6, 3, 5, 2, 4])

In [67]:
df.take(column_sampler, axis="columns")

,0,1,6,3,5,2,4
0,0,1,6,3,5,2,4
1,7,8,13,10,12,9,11
2,14,15,20,17,19,16,18
3,21,22,27,24,26,23,25
4,28,29,34,31,33,30,32


To select a random subset without replacement (the same row cannot appear twice), you can use the `sample` method on Series and DataFrame:



In [68]:
df.sample(n=3)

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
4,28,29,30,31,32,33,34
3,21,22,23,24,25,26,27


To generate a sample with replacement (to allow repeat choices), pass `replace=True` to `sample`:



In [69]:
choices = pd.Series([5, 7, -1, 6, 4])
choices.sample(n=10, replace=True)


2   -1
4    4
2   -1
3    6
1    7
2   -1
2   -1
4    4
2   -1
3    6
dtype: int64

## Computing Indicator/Dummy Variables
Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a dummy or indicator matrix. If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s. pandas has a `pandas.get_dummies` function for doing this, though you could also devise one yourself. Let’s consider an example DataFrame:



In [70]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [71]:
pd.get_dummies(df["key"])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data. `pandas.get_dummies` has a prefix argument for doing this:



In [72]:
dummies = pd.get_dummies(df["key"], prefix="key")
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


The `DataFrame.join` method will be explained in more detail in the next chapter.

If a row in a DataFrame belongs to multiple categories, we have to use a different approach to create the dummy variables. Let’s look at the MovieLens 1M dataset, which is investigated in more detail in [Ch 13: Data Analysis Examples](https://wesmckinney.com/book/data-cleaning.html#data-analysis-examples):



In [73]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("datasets/movielens/movies.dat", sep="::",
                       header=None, names=mnames, engine="python")
movies[:10]

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/movielens/movies.dat'

pandas has implemented a special Series method `str.get_dummies` (methods that start with `str`. are discussed in more detail later in [String Manipulation](https://wesmckinney.com/book/data-cleaning.html#text_string_manip)) that handles this scenario of multiple group membership encoded as a delimited string:



In [ ]:
dummies = movies["genres"].str.get_dummies("|")
dummies.iloc[:10, :6]


Then, as before, you can combine this with `movies` while adding a `"Genre_"` to the column names in the `dummies` DataFrame with the `add_prefix` method:



In [ ]:
ovies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic.iloc[0]

- For much larger data, this method of constructing indicator variables with multiple membership is not especially speedy. It would be better to write a lower-level function that writes directly to a NumPy array, and then wrap the result in a DataFrame.

A useful recipe for statistical applications is to combine `pandas.get_dummies` with a discretization function like `pandas.cut`:

In [ ]:
np.random.seed(12345) # to make the example repeatable
values = np.random.uniform(size=10)
values

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

We will look again at `pandas.get_dummies` later in [Creating dummy variables for modeling](https://wesmckinney.com/book/data-cleaning.html#pandas-categorical-dummy).

# Extension Data Types

- This is a newer and more advanced topic that many pandas users do not need to know a lot about, but I present it here for completeness since I will reference and use extension data types in various places in the upcoming chapters.

pandas was originally built upon the capabilities present in NumPy, an array computing library used primarily for working with numerical data. Many pandas concepts, such as missing data, were implemented using what was available in NumPy while trying to maximize compatibility between libraries that used NumPy and pandas together.

Building on NumPy led to a number of shortcomings, such as:

* Missing data handling for some numerical data types, such as integers and Booleans, was incomplete. As a result, when missing data was introduced into such data, pandas converted the data type to `float64` and used `np.nan` to represent null values. This had compounding effects by introducing subtle issues into many pandas algorithms.


* Datasets with a lot of string data were computationally expensive and used a lot of memory.


* Some data types, like time intervals, timedeltas, and timestamps with time zones, could not be supported efficiently without using computationally expensive arrays of Python objects.

More recently, pandas has developed an extension type system allowing for new data types to be added even if they are not supported natively by NumPy. These new data types can be treated as first class alongside data coming from NumPy arrays.

Let's look at an example where we create a Series of integers with a missing value:





In [ ]:
s = pd.Series([1, 2, 3, None])
s

In [ ]:
s.dtype

Mainly for backward compatibility reasons, Series uses the legacy behavior of using a `float64` data type and `np.nan` for the missing value. We could create this Series instead using `pandas.Int64Dtype`:



In [ ]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
s

In [ ]:
s.isna()

In [ ]:
s.dtype

The output `<NA>` indicates that a value is missing for an extension type array. This uses the special `pandas.NA` sentinel value:



In [ ]:
s[3]

In [ ]:
s[3] is pd.NA

We also could have used the shorthand `"Int64"` instead of `pd.Int64Dtype()` to specify the type. The capitalization is necessary, otherwise it will be a NumPy-based nonextension type:



In [ ]:
s = pd.Series([1, 2, 3, None], dtype="Int64")

pandas also has an extension type specialized for string data that does not use NumPy object arrays (it requires the pyarrow library, which you may need to install separately):

In [ ]:
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s


These string arrays generally use much less memory and are frequently computationally more efficient for doing operations on large datasets.

Another important extension type is `Categorical`, which we discuss in more detail in [Categorical Data](https://wesmckinney.com/book/data-cleaning.html#pandas-categorical). A reasonably complete list of extension types available as of this writing is in [table_title](https://wesmckinney.com/book/data-cleaning.html#table_pandas_extension_types).

Extension types can be passed to the Series `astype` method, allowing you to convert easily as part of your data cleaning process:


In [ ]:
df = pd.DataFrame({"A": [1, 2, None, 4],
                   "B": ["one", "two", "three", None],
                   "C": [False, None, False, True]})
df

In [ ]:
df["A"] = df["A"].astype("Int64")
df["B"] = df["B"].astype("string")
df["C"] = df["C"].astype("boolean")
df



There are a lot more types, found in the [chapter](https://wesmckinney.com/book/data-cleaning.html#pandas-ext-types).

# String Manipulation

Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing. Most text operations are made simple with the string object’s built-in methods. For more complex pattern matching and text manipulations, regular expressions may be needed. pandas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data.

## Python Built-In String Object Methods
In many string munging and scripting applications, built-in string methods are sufficient. As an example, a comma-separated string can be broken into pieces with `split`:



In [ ]:
val = "a,b,  guido"
val.split(",")

`split` is often combined with `strip` to trim whitespace (including line breaks):



In [ ]:
pieces = [x.strip() for x in val.split(",")]
pieces

These substrings could be concatenated together with a two-colon delimiter using addition:

In [ ]:
first, second, third = pieces
first + "::" + second + "::" + third


But this isn’t a practical generic method. A faster and more Pythonic way is to pass a list or tuple to the `join` method on the string `"::"`:



In [ ]:
"::".join(pieces)

Other methods are concerned with locating substrings. Using Python’s `in` keyword is the best way to detect a substring, though `index` and `find` can also be used:



In [ ]:
val # To show the value where are manipulating

In [ ]:
"guido" in val

In [ ]:
val.index(",") # If the value wasn't there this would return an error.

In [ ]:
val.find(":") # It's not found so it returns -1

Relatedly, `count` returns the number of occurrences of a particular substring:

In [ ]:
val.count(",")

`replace` will substitute occurrences of one pattern for another. It is commonly used to delete patterns, too, by passing an empty string:



In [ ]:
val # To show what value we are manipulating

In [ ]:
val.replace(",", "::")

In [ ]:
val.replace(",", "")

See [Table 7.4](https://wesmckinney.com/book/data-cleaning.html#tbl-table_string_methods) for a listing of some of Python's string methods.

Regular expressions can also be used with many of these operations, as you’ll see.

## Regular Expressions

Yes we can use these too.

The Python `re` module functions fall into three categories: pattern matching, substitution, and splitting. Naturally these are all related; a regex describes a pattern to locate in the text, which can then be used for many purposes. Let’s look at a simple example: suppose we wanted to split a string with a variable number of whitespace characters (tabs, spaces, and newlines).

The regex describing one or more whitespace characters is `\s+`:



In [ ]:
import re
text = "foo    bar\t baz  \tqux"
re.split(r"\s+", text)

When you call `re.split(r"\s+", text)`, the regular expression is first compiled, and then its `split` method is called on the passed text. You can compile the regex yourself with `re.compile`, forming a reusable regex object:

In [ ]:
regex = re.compile(r"\s+")
regex.split(text)

If, instead, you wanted to get a list of all patterns matching the regex, you can use the `findall` method:



In [ ]:
regex.findall(text)

- To avoid unwanted escaping with \ in a regular expression, use raw string literals like `r"C:\x"` instead of the equivalent `"C:\\x"`.


Creating a regex object with re.compile is highly recommended if you intend to apply the same expression to many strings; doing so will save CPU cycles.

`match` and `search` are closely related to `findall`. While `findall` returns all matches in a string, `search` returns only the first match. More rigidly, `match` only matches at the beginning of the string. As a less trivial example, let’s consider a block of text and a regular expression capable of identifying most email addresses:




In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

# re.IGNORECASE makes the regex case insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

Using `findall` on the text produces a list of the email addresses:



In [ ]:
regex.findall(text)

`search` returns a special match object for the first email address in the text. For the preceding regex, the match object can only tell us the start and end position of the pattern in the string:

In [ ]:
m = regex.search(text)
m

In [ ]:
text[m.start():m.end()]

`regex.match` returns `None`, as it will match only if the pattern occurs at the start of the string:

In [ ]:
print(regex.match(text))

Relatedly, `sub` will return a new string with occurrences of the pattern replaced by a new string:



In [ ]:
print(regex.sub("REDACTED", text))

Suppose you wanted to find email addresses and simultaneously segment each address into its three components: username, domain name, and domain suffix. To do this, put parentheses around the parts of the pattern to segment:



In [ ]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
regex = re.compile(pattern, flags=re.IGNORECASE)

A match object produced by this modified regex returns a tuple of the pattern components with its `groups` method:



In [ ]:
m = regex.match("wesm@bright.net")
m.groups()

`findall` returns a list of tuples when the pattern has groups:

In [ ]:
regex.findall(text)

`sub` also has access to groups in each match using special symbols like `\1` and `\2`. The symbol `\1` corresponds to the first matched group, `\2` corresponds to the second, and so forth:



In [ ]:
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

There is much more to regular expressions in Python, most of which is outside the book’s scope. [Table 7.5](https://wesmckinney.com/book/data-cleaning.html#tbl-table_regex_method) provides a brief summary.


## String Functions in pandas

Cleaning up a messy dataset for analysis often requires a lot of string manipulation. To complicate matters, a column containing strings will sometimes have missing data:





In [74]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}
data = pd.Series(data)
data


Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [75]:
data.isna()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

String and regular expression methods can be applied (passing a `lambda` or other function) to each value using `data.map`, but it will fail on the NA (null) values. To cope with this, Series has array-oriented methods for string operations that skip over and propagate NA values. These are accessed through Series’s `str` attribute; for example, we could check whether each email address has `"gmail"` in it with `str.contains`:



In [76]:
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Note that the result of this operation has an `object` dtype. pandas has extension types that provide for specialized treatment of strings, integers, and Boolean data which until recently have had some rough edges when working with missing data:



In [77]:
data_as_string_ext = data.astype('string')
data_as_string_ext

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [78]:
data_as_string_ext.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

Extension types are discussed in more detail in [Extension Data Types](https://wesmckinney.com/book/data-cleaning.html#pandas-ext-types).


Regular expressions can be used, too, along with any re options like `IGNORECASE`:



In [80]:
import re
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

There are a couple of ways to do vectorized element retrieval. Either use `str.get` or index into the `str` attribute:



In [81]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [82]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

You can similarly slice strings using this syntax:

In [83]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

The `str.extract` method will return the captured groups of a regular expression as a DataFrame:



In [84]:
data.str.extract(pattern, flags=re.IGNORECASE)


,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


See [Table 7.6](https://wesmckinney.com/book/data-cleaning.html#tbl-table_vec_string) for more pandas string methods.



# 7.5 Categorical Data
This section introduces the pandas `Categorical` type. I will show how you can achieve better performance and memory use in some pandas operations by using it. I also introduce some tools that may help with using categorical data in statistics and machine learning applications.

## Background and Motivation
Frequently, a column in a table may contain repeated instances of a smaller set of distinct values. We have already seen functions like `unique` and `value_counts`, which enable us to extract the distinct values from an array and compute their frequencies, respectively:



In [85]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [86]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [87]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

Many data systems (for data warehousing, statistical computing, or other uses) have developed specialized approaches for representing data with repeated values for more efficient storage and computation. In data warehousing, a best practice is to use so-called _dimension tables_ containing the distinct values and storing the primary observations as integer keys referencing the dimension table:



In [88]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [89]:
dim

0     apple
1    orange
dtype: object

We can use the `take` method to restore the original Series of strings:



In [90]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

This representation as integers is called the categorical or dictionary-encoded representation. The array of distinct values can be called the _categories, dictionary_, or _levels_ of the data. In this book we will use the terms _categorical_ and _categories_. The integer values that reference the categories are called the _category codes_ or simply codes.

The categorical representation can yield significant performance improvements when you are doing analytics. You can also perform transformations on the categories while leaving the codes unmodified. Some example transformations that can be made at relatively low cost are:

* Renaming categories


* Appending a new category without changing the order or position of the existing categories



## Categorical Extension Type in pandas
pandas has a special `Categorical` extension type for holding data that uses the integer-based categorical representation or encoding. This is a popular data compression technique for data with many occurrences of similar values and can provide significantly faster performance with lower memory use, especially for string data.

Let's consider the example Series from before:

